In [ ]:
import os
import logging
import pandas as pd
import mysql.connector as sql
import matplotlib.pyplot as plt
from IPython.display import display

logging.basicConfig(
    filename="activity.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def get_connection():
    return sql.connect(
        host="localhost",
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        database="fifa"
    )

# One-Time Setup
def setup_database(csv_path="players.csv"):
    #One-time setup:- Creates table if it does not exist,Loads initial CSV data
    conn = get_connection()
    cursor = conn.cursor()
    try:
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS info (
            short_name VARCHAR(50) PRIMARY KEY,
            age INT,
            height_cm INT,
            weight_kg INT,
            nationality VARCHAR(50),
            club VARCHAR(60),
            wage INT,
            player_position VARCHAR(30),
            body_type VARCHAR(20),
            team_position CHAR(4),
            team_jersey_number INT,
            pace INT
        )
        """)

        df = pd.read_csv(csv_path)

        query = """INSERT IGNORE INTO info VALUES
        (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""

        for _, row in df.iterrows():
            cursor.execute(query, tuple(row))

        conn.commit()
        print("Database setup completed")

    except Exception:
        conn.rollback()
        print("Setup failed")
    finally:
        conn.close()

# Input helpers
def safe_int(prompt):
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("Please enter a valid number")

def safe_str(prompt):
    val = input(prompt).strip()
    return val if val else safe_str(prompt)

# Menus
def mainmenu():
    print("""
―――――――――――――――――――――――――――――――――――
‖ 1. Data Collection                ‖
‖ 2. Data Manipulation              ‖
‖ 3. Data Analysis                  ‖
‖ 4. Data Visualization             ‖
‖ 5. Export to CSV                  ‖
‖ 6. Exit                           ‖
―――――――――――――――――――――――――――――――――――
""")

def exit_program():
    print("Thank you for your response")

# Data Collection
def show_data():
    conn = get_connection()
    df = pd.read_sql("SELECT * FROM info", conn)
    display(df)
    conn.close()

# Insert
def insert_player():
    conn = get_connection()
    cursor = conn.cursor()

    data = (
        safe_str("Name: "),
        safe_int("Age: "),
        safe_int("Height (cm): "),
        safe_int("Weight (kg): "),
        safe_str("Nationality: "),
        safe_str("Club: "),
        safe_int("Wage: "),
        safe_str("Player Position: "),
        safe_str("Body Type: "),
        safe_str("Team Position: "),
        safe_int("Jersey Number: "),
        safe_int("Pace: ")
    )

    query = """INSERT INTO info VALUES
    (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""

    cursor.execute(query, data)
    conn.commit()
    logging.info(f"Inserted player {data[0]}")
    print("Player inserted successfully")

    conn.close()

# Delete
def delete_player():
    conn = get_connection()
    cursor = conn.cursor()
    name = safe_str("Enter player name to delete: ")

    confirm = input("Type YES to confirm: ")
    if confirm != "YES":
        print("Deletion cancelled")
        return

    cursor.execute("DELETE FROM info WHERE short_name=%s", (name,))
    conn.commit()
    logging.warning(f"Deleted player {name}")
    print("Player deleted")

    conn.close()

# Update
def update_player():
    conn = get_connection()
    cursor = conn.cursor()

    name = safe_str("Player name: ")
    field = safe_str("Field (age, club, wage, pace): ")
    value = safe_str("New value: ")

    allowed = {"age", "club", "wage", "pace"}
    if field not in allowed:
        print("Invalid field")
        return

    cursor.execute(
        f"UPDATE info SET {field}=%s WHERE short_name=%s",
        (value, name)
    )
    conn.commit()
    logging.info(f"Updated {field} for {name}")
    print("Player updated")

    conn.close()

# Analysis
def analysis_menu():
    print("""
――――――――――――――――――――――――――――――――――――――
‖ 1. Display top records                ‖
‖ 2. Display bottom records             ‖
‖ 3. Display a particular player record ‖
‖ 4. Display records based on condition ‖
‖ 5. Go back to main menu               ‖
――――――――――――――――――――――――――――――――――――――
""")

def analyze_data():
    conn = get_connection()

    while True:
        analysis_menu()
        choice = safe_int("Enter your choice: ")

        # 1.Top records
        if choice == 1:
            n = safe_int("Enter number of top rows: ")
            df = pd.read_sql("SELECT * FROM info LIMIT %s", conn, params=[n])
            display(df)

        # 2.Bottom records
        elif choice == 2:
            n = safe_int("Enter number of bottom rows: ")
            df = pd.read_sql(
                "SELECT * FROM info ORDER BY short_name DESC LIMIT %s",
                conn,
                params=[n]
            )
            display(df)

        # 3.Particular player
        elif choice == 3:
            name = safe_str("Enter player name: ")
            df = pd.read_sql(
                "SELECT * FROM info WHERE short_name = %s",
                conn,
                params=[name]
            )
            display(df)

        # 4.Conditional filtering
        elif choice == 4:
            print("""
1. Filter by team jersey number
2. Filter by age
3. Filter by nationality
4. Filter by club
""")
            cond = safe_int("Enter your choice: ")

            if cond == 1:
                start = safe_int("Starting jersey number: ")
                end = safe_int("Ending jersey number: ")
                df = pd.read_sql(
                    "SELECT * FROM info WHERE team_jersey_number BETWEEN %s AND %s",
                    conn,
                    params=[start, end]
                )
                display(df)

            elif cond == 2:
                start = safe_int("Starting age: ")
                end = safe_int("Ending age: ")
                df = pd.read_sql(
                    "SELECT * FROM info WHERE age BETWEEN %s AND %s",
                    conn,
                    params=[start, end]
                )
                display(df)

            elif cond == 3:
                country = safe_str("Enter nationality: ")
                df = pd.read_sql(
                    "SELECT * FROM info WHERE nationality = %s",
                    conn,
                    params=[country]
                )
                display(df)

            elif cond == 4:
                club = safe_str("Enter club name: ")
                df = pd.read_sql(
                    "SELECT * FROM info WHERE club = %s",
                    conn,
                    params=[club]
                )
                display(df)

        # 5.Back
        elif choice == 5:
            break

        else:
            print("Invalid choice")

    conn.close()

# Visualisation
def visualize_data():
    conn = get_connection()
    df = pd.read_sql("SELECT * FROM info", conn)
    conn.close()

    print("""
―――――――――――――――――――――――――
‖ 1. Line Graphs         ‖
‖ 2. Bar Graphs          ‖
‖ 3. Histogram Graphs    ‖
‖ 4. Back                ‖
―――――――――――――――――――――――――
""")

    g = safe_int("Enter your choice: ")

    # Line Graph
    if g == 1:
        print("""
a. Player vs Height
b. Player vs Pace
c. Player vs Body Type
""")
        ch = input("Enter choice: ")

        if ch == 'a':
            plt.plot(
                df["short_name"].head(20),
                df["height_cm"].head(20),
                marker="*",
                color="orange"
            )
            plt.grid()
            plt.xlabel("player")
            plt.ylabel("height")
            plt.xticks(rotation="vertical")
            plt.title("player vs height")
            plt.show()

        elif ch == 'b':
            plt.plot(
                df["short_name"].head(20),
                df["pace"].head(20),
                marker="*",
                color="m"
            )
            plt.grid()
            plt.xlabel("player")
            plt.ylabel("pace")
            plt.xticks(rotation="vertical")
            plt.title("player vs pace")
            plt.show()

        elif ch == 'c':
            plt.plot(
                df["short_name"].tail(20),
                df["body_type"].head(20),
                marker="*",
                color="c"
            )
            plt.grid()
            plt.xlabel("player")
            plt.ylabel("body type")
            plt.xticks(rotation="vertical")
            plt.title("player vs body type")
            plt.show()

    # Bar Graph
    elif g == 2:
        print("""
d. Age vs Player Count
e. Players vs Country
f. Players vs Club
""")
        ch = input("Enter choice: ")

        if ch == 'd':
            temp = df.groupby("age").size()
            plt.bar(
                temp.index,
                temp.values,
                color="red",
                edgecolor="black"
            )
            plt.xlabel("age")
            plt.ylabel("number of players")
            plt.title("player vs age")
            plt.show()

        elif ch == 'e':
            temp = df.groupby("nationality").size()
            plt.bar(
                temp.index,
                temp.values,
                color="yellow",
                edgecolor="black"
            )
            plt.xticks(rotation="vertical")
            plt.xlabel("country")
            plt.ylabel("player")
            plt.title("player vs country")
            plt.show()

        elif ch == 'f':
            temp = df.groupby("club").size()
            plt.bar(
                temp.index,
                temp.values,
                color="orange",
                edgecolor="black"
            )
            plt.xticks(rotation="vertical")
            plt.xlabel("club")
            plt.ylabel("number of player")
            plt.title("player vs club")
            plt.show()

    # Histogram
    elif g == 3:
        print("""
g. Wage Earned
h. Weight (kg)
""")
        ch = input("Enter choice: ")

        if ch == 'g':
            plt.hist(
                df["wage"],
                bins=[0,100000,200000,300000,400000,500000,600000],
                color="orange",
                edgecolor="black"
            )
            plt.xlabel("wage(in euros)")
            plt.ylabel("number of players")
            plt.title("wage earned")
            plt.show()

        elif ch == 'h':
            plt.hist(
                df["weight_kg"],
                bins=[50,60,70,80,90,100],
                histtype="barstacked",
                color="y",
                edgecolor="black"
            )
            plt.xlabel("weight(in kg)")
            plt.ylabel("number of players")
            plt.title("weight(in kg)")
            plt.show()

    elif g == 4:
        return


# Export
def export_csv():
    conn = get_connection()
    df = pd.read_sql("SELECT * FROM info", conn)
    df.to_csv("players_export.csv", index=False)
    conn.close()
    print("Data exported to players_export.csv")

# Main Driver
def run():
    print("Hello user! Welcome.")
    mainmenu()

    while True:
        choice = safe_int("Enter your choice: ")

        if choice == 1:
            show_data()
        elif choice == 2:
            insert_player()
        elif choice == 3:
            analyze_data()
        elif choice == 4:
            visualize_data()
        elif choice == 5:
            export_csv()
        elif choice == 6:
            exit_program()
            break
        else:
            print("Invalid choice")

# Start
if __name__ == "__main__":
    run()
